<a href="https://colab.research.google.com/github/massinoLight/tp_AlgoBD/blob/main/TP_Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 51.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=daa68dc8ee00d626516d5f40bfa1aaaa52c2c3d4e982c5b59272d93ed6cf3362
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


##On charge le fichier de data set

In [2]:
import requests 
file_url = "https://raw.githubusercontent.com/apache/spark/master/data/mllib/kmeans_data.txt"
	
r = requests.get(file_url, stream = True) 

with open("/content/sample_data/kmeans_data.txt", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

##Spark context

In [3]:
from numpy import array
from math import sqrt

from pyspark import SparkContext
from pyspark.mllib.clustering import KMeans, KMeansModel


sc = SparkContext(appName="KMeansExample")  

##Determiner notre modéle (nb cluster=3,nb iteration=10)

In [6]:
data = sc.textFile("/content/sample_data/kmeans_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

clusters = KMeans.train(parsedData, 3, maxIterations=10, initializationMode="random")

def error(point):
        center = clusters.centers[clusters.predict(point)]
        return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

    

    



Within Set Sum of Squared Error = 0.519615242270665


In [ ]:
clusters.save(sc, "model/Kmeans/KMeansModel")
sameModel = KMeansModel.load(sc, "model/Kmeans/KMeansModel")